In [3]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.cluster import KMeans, k_means_, DBSCAN, AgglomerativeClustering
from sklearn import cluster
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_distances

import hdbscan
from gensim import models
from gensim.corpora import mmcorpus, Dictionary
from gensim.models import lsimodel, ldamodel, tfidfmodel, rpmodel, logentropy_model, TfidfModel, LsiModel, LdaModel
from gensim import matutils, corpora
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.svm import SVC
import sys
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import html
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import chat_bot as cb
import Database_Handler as dh
from datetime import datetime
import seaborn as sns
import functools
from konlpy.tag import Mecab
from konlpy.utils import pprint
mecab = Mecab()
%matplotlib inline

In [4]:
dataDict = pickle.load(open('./data/pre_data/stastics/for_statistics_Naver_from_mongodb.pickled','rb'))
print (len(dataDict))

7681


In [5]:
keywordsDict = pickle.load(open('./data/pre_data/keywords/keywords_Naver.pickled','rb'))

In [6]:
for idx in dataDict:
    dataDict[idx]['extracted_keywords'] = keywordsDict[idx]

In [7]:
df = pd.DataFrame.from_dict(dataDict, orient='index')
df['date'] = pd.to_datetime(df['date']).dt.date
df.reset_index(inplace = True)
df.rename(columns={'index':'id'}, inplace=True)
print (df.shape)
df.head()

(7681, 11)


,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a29c445588c132954d1973a,정치,2017-12-07,연합뉴스,1713,1474,1,"北외무성 ""전쟁 바라지 않지만 결코 피하지 않을 것""","美고위인사 대북언급 비난하며 ""전쟁 기정사실화"" 위협 며칠 새 이어지는 북한 군민...","[외무성, 핵전쟁, 대변인]","{고위, 핵전쟁, 북한, 도화선, 중앙, 조선반도, 미국, 대변인}"
1,5a29c445588c132954d1973b,정치,2017-12-07,한국일보,2551,2121,2,"예산전쟁, 예결위 간사ㆍ호남이 웃었다",예결위 간사들이 최대 수혜자..당 지도부 내 몫 챙기기도 여전 황주홍ㆍ김도읍 등...,"[예산, 예결위, soc]","{예산안, 증액, 호남, 의원, 국민의당, 지역구, 정부안}"
2,5a29c445588c132954d1973c,정치,2017-12-07,뉴시스,610,540,3,"혐의 부인에 20시간 조사…檢, 최경환 구속 카드 꺼내나",【서울=뉴시스】 최진석 기자 = 박근혜 정부 시절 국가정보원 특수활동비 수수 의혹 ...,"[최경환, 구속영장, 국가정보원]","{검찰, 국정원장, 구속영장 청구, 의원, 혐의, 정기국회, 조사}"
3,5a29c445588c132954d1973d,정치,2017-12-07,연합뉴스,145,134,4,"최재형 감사원장 후보자 ""독립성 강화는 임명권자의 뜻""",감사원장에 내정된 최재형 사법연수원장(고양=연합뉴스) 이희열 기자 = 7일 감사원장...,"[이슈 · 최재형 감사원장 내정, 감사원장, 최재형, 감사원]","{법관, 공직 사회, 감사원장, 후보자, 지명, 생활}"
4,5a29c445588c132954d1973e,정치,2017-12-07,동아일보,1074,937,5,"B-1B 한반도에 뜨자, 평양 비운 김정은",[동아일보] 북중 접경지 양강도 삼지연 시찰… 방북 유엔 사무차장 면담 안할듯 B-...,"[김정은, b-1b, 한반도]","{시찰, 펠트먼, 접경, 양강도, 사무차장, 공장, 삼지연, 훈련, 김정은}"


In [8]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

In [9]:
def nav_tokenizer(corpus):
    pos = mecab.pos(corpus)
    pos = ['/'.join(t) for t in pos if not t[0] in stopwords]
    return pos

In [10]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

In [31]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['category'])
le.classes_

array(['IT/과학', '경제', '사회', '생활/문화', '세계', '스포츠', '연예', '정치'],
      dtype=object)

In [12]:
w2v_docs = list()
for idx in tqdm(df.index):
    text = df.loc[idx,'title']+'.\n'+df.loc[idx,'mainText']
    pos = nav_tokenizer(text)
    label = df.loc[idx, 'category']
    label = le.transform([label])
    w2v_docs.append(TaggedDocument(pos, label))

100%|██████████| 7681/7681 [13:45<00:00,  9.30it/s]


In [13]:
train, test = train_test_split(w2v_docs, test_size = 0.15)

## doc2vec

In [34]:
d2v_model = Doc2Vec(alpha=0.025, min_alpha=0.025, iter=10)
d2v_model.build_vocab(tqdm(w2v_docs))
for epoch in tqdm(range(10)):
    d2v_model.train(tqdm(train),total_examples=d2v_model.corpus_count, epochs=d2v_model.iter)
    d2v_model.alpha -= 0.002 
    d2v_model.min_alpha = d2v_model.alpha

 37%|███▋      | 2441/6528 [00:01<00:03, 1222.96it/s]


100%|██████████| 10/10 [07:36<00:00, 45.67s/it]it/s]


In [43]:
d2v_model.save('./model/keywords_classification_naver_by_doc2vec_size100_window5_iter10_by_mecab.model')

In [14]:
d2v_model = Doc2Vec.load('./model/keywords_classification_naver_by_doc2vec_size100_window5_iter10_by_mecab.model')

In [26]:
x_train = [x.words for x in train]
y_train = [x.tags for x in train]
x_test = [x.words for x in test]
y_test = [x.tags for x in test]


5767it [03:12, 29.93it/s]

In [47]:
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([d2v_model.infer_vector(z).reshape(1,-1) for z in tqdm(map(lambda x: x, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([d2v_model.infer_vector(z).reshape(1,-1) for z in tqdm(map(lambda x: x, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)



0it [00:00, ?it/s]

1it [00:00,  5.31it/s]

6it [00:00, 18.67it/s]

11it [00:00, 25.33it/s]

18it [00:00, 24.49it/s]

24it [00:00, 28.59it/s]

28it [00:00, 29.78it/s]

32it [00:01, 30.68it/s]

38it [00:01, 32.74it/s]

44it [00:01, 32.95it/s]

48it [00:01, 31.34it/s]

52it [00:01, 29.42it/s]

58it [00:01, 29.66it/s]

65it [00:02, 31.57it/s]

70it [00:02, 32.23it/s]

77it [00:02, 33.71it/s]

82it [00:02, 33.27it/s]

87it [00:02, 31.99it/s]

91it [00:02, 32.26it/s]

95it [00:03, 30.58it/s]

98it [00:03, 30.51it/s]

101it [00:03, 30.37it/s]

107it [00:03, 31.06it/s]

111it [00:03, 31.03it/s]

115it [00:03, 31.12it/s]

120it [00:03, 31.23it/s]

124it [00:03, 31.44it/s]

128it [00:04, 31.45it/s]

133it [00:04, 31.73it/s]

137it [00:04, 31.39it/s]

143it [00:04, 32.01it/s]

148it [00:04, 32.25it/s]

153it [00:04, 31.46it/s]

158it [00:04, 31.71it/s]

162it [00:05, 31.68it/s]

167it [00:05, 31.89it/s]

172it [00:05, 32.12it/s]

176it [00:05, 32.11it/s]

182it [00:05, 32.59it/s]

187it [00:05

1771it [00:40, 43.87it/s]
1784it [00:40, 44.09it/s]
1798it [00:40, 44.32it/s]
1815it [00:40, 44.63it/s]
1828it [00:40, 44.77it/s]
1840it [00:40, 44.94it/s]
1852it [00:41, 45.10it/s]
1866it [00:41, 45.33it/s]
1884it [00:41, 45.65it/s]
1898it [00:41, 45.81it/s]
1915it [00:41, 46.11it/s]
1929it [00:41, 46.31it/s]
1948it [00:41, 46.65it/s]
1963it [00:41, 46.89it/s]
1978it [00:41, 47.13it/s]
1993it [00:42, 47.37it/s]
2008it [00:42, 47.60it/s]
2022it [00:42, 47.81it/s]
2036it [00:42, 47.98it/s]
2049it [00:42, 48.16it/s]
2066it [00:42, 48.44it/s]
2086it [00:42, 48.80it/s]
2102it [00:42, 49.03it/s]
2117it [00:42, 49.26it/s]
2132it [00:43, 49.46it/s]
2146it [00:43, 49.63it/s]
2160it [00:43, 49.83it/s]
2175it [00:43, 50.06it/s]
2189it [00:43, 50.23it/s]
2202it [00:43, 50.42it/s]
2215it [00:43, 50.55it/s]
2230it [00:43, 50.77it/s]
2245it [00:44, 50.99it/s]
2258it [00:44, 51.13it/s]
2270it [00:44, 51.26it/s]
2282it [00:44, 51.35it/s]
2293it [00:44, 51.48it/s]
2306it [00:44, 51.65it/s]
2323it [00:4

3977it [01:24, 46.96it/s]
3981it [01:24, 46.94it/s]
3985it [01:25, 46.86it/s]
3989it [01:25, 46.85it/s]
3997it [01:25, 46.89it/s]
4004it [01:25, 46.91it/s]
4010it [01:25, 46.92it/s]
4019it [01:25, 46.97it/s]
4026it [01:25, 46.99it/s]
4035it [01:25, 47.03it/s]
4042it [01:25, 47.01it/s]
4048it [01:26, 47.02it/s]
4054it [01:26, 46.97it/s]
4059it [01:26, 46.91it/s]
4064it [01:26, 46.87it/s]
4068it [01:26, 46.86it/s]
4072it [01:26, 46.84it/s]
4076it [01:27, 46.79it/s]
4080it [01:27, 46.76it/s]
4090it [01:27, 46.82it/s]
4096it [01:27, 46.80it/s]
4101it [01:27, 46.78it/s]
4106it [01:27, 46.75it/s]
4110it [01:28, 46.64it/s]
4114it [01:28, 46.55it/s]
4117it [01:28, 46.46it/s]
4121it [01:28, 46.43it/s]
4125it [01:28, 46.41it/s]
4128it [01:29, 46.38it/s]
4132it [01:29, 46.36it/s]
4136it [01:29, 46.27it/s]
4139it [01:29, 46.23it/s]
4142it [01:29, 46.19it/s]
4145it [01:29, 46.13it/s]
4149it [01:29, 46.12it/s]
4153it [01:30, 46.10it/s]
4157it [01:30, 46.09it/s]
4161it [01:30, 46.08it/s]
4165it [01:3

1153it [00:12, 93.78it/s]


In [48]:
from keras.utils import np_utils
y_test2 = np_utils.to_categorical(y_test,8)
y_train2 = np_utils.to_categorical(y_train,8)

In [49]:
train_vecs_w2v[1]

array([-0.8149893 , -0.5927798 , -0.35110688, -0.8460614 ,  0.8934493 ,
       -0.12694106,  0.9674166 ,  0.03004529,  0.82302374,  0.14543392,
        0.9668528 ,  0.08554414, -0.16445161,  0.0843067 ,  0.46871617,
       -0.95718604,  0.20431744,  0.7897833 , -1.4506617 , -0.41610292,
        0.47897878,  0.6144561 , -0.0457495 ,  0.5283465 ,  0.17395525,
        0.39388067,  0.23954777,  0.21546623, -0.5062416 ,  0.8503014 ,
       -1.2265496 , -1.1485617 ,  0.48467326,  0.69670534, -0.32025516,
        0.04358882,  1.0795076 ,  0.6783447 , -1.732311  ,  0.6530629 ,
       -0.07607748,  0.61339337, -0.07083827, -0.09134725,  0.69040304,
       -0.8678795 , -0.6697021 , -0.01280149, -1.107479  , -0.5038705 ,
       -0.07053655,  1.3977698 ,  0.06045599,  0.0891863 ,  0.04556778,
       -0.79224426, -0.94056845, -0.6247414 ,  0.05557533,  0.68172437,
        0.15707673, -0.5591665 , -0.7312885 , -0.69578683, -0.07875763,
       -0.6939183 ,  1.9375011 , -0.3092126 , -1.5068412 , -0.21

In [51]:
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Dense, Embedding, embeddings, merge, Dropout
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(16, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(8, activation='sigmoid'))
model.compile(#optimizer='rmsprop',
    optimizer='adadelta',
              #loss='binary_crossentropy',
    #optimizer=SGD(lr=0.2), 
    loss='categorical_crossentropy',
    metrics=['accuracy'])

model.fit(train_vecs_w2v, y_train2, epochs=100, verbose=0, callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])
score = model.evaluate(test_vecs_w2v, y_test2, verbose=2)
print (score[1])

HBox(children=(IntProgress(value=0, description='Training'), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 1', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 2', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 3', max=6528), HTML(value='')))

...

HBox(children=(IntProgress(value=0, description='Epoch 96', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 97', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 98', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 99', max=6528), HTML(value='')))


0.9609713790112749


In [70]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
model2 = Sequential()
model2.add(Dense(64, activation='relu', input_dim=100))
model2.add(Dropout(0.25))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.125))
model2.add(Dense(8, activation='softmax'))
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model2.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
model2.fit(train_vecs_w2v, y_train2,
          epochs=100, verbose=0,
          callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])
score = model2.evaluate(test_vecs_w2v, y_test2)
print (score[1])

HBox(children=(IntProgress(value=0, description='Training'), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 1', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 2', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 3', max=6528), HTML(value='')))

...

HBox(children=(IntProgress(value=0, description='Epoch 96', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 97', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 98', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 99', max=6528), HTML(value='')))


1153/1153 [==============================] - 0s 111us/step
0.9635732870771899
